In [1]:
!pip install accelerate
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

import pandas as pd

# Correcting the path to use the full path from the root
data_path = "/content/drive/MyDrive/nlp final project/AnonymizedClinicalAbbreviationsAndAcronymsDataSet.txt"

try:
    data = pd.read_csv(data_path, sep='|', header=None, names=[
        "Short_form", "Long_form", "Abbreviation", "Start_pos", "End_pos", "Section_info", "Context"
    ], encoding='ISO-8859-1')
except UnicodeDecodeError:
    data = pd.read_csv(data_path, sep='|', header=None, names=[
        "Short_form", "Long_form", "Abbreviation", "Start_pos", "End_pos", "Section_info", "Context"
    ], encoding='ISO-8859-1', errors='ignore')

print(data.head())
print(data.info())


Mounted at /content/drive/
  Short_form Long_form Abbreviation  Start_pos  End_pos  \
0         AB  abortion          AB.      231.0    233.0   
1         AB  abortion          AB.      249.0    251.0   
2         AB  abortion           AB      223.0    224.0   
3         AB  abortion          AB.      194.0    196.0   
4         AB  abortion           AB      114.0    115.0   

                     Section_info  \
0                             NaN   
1                             NaN   
2                 PAST OB HISTORY   
3  HISTORY OF THE PRESENT ILLNESS   
4             PAST OB-GYN HISTORY   

                                             Context  
0  _%#NAME#%_ _%#NAME#%_ is a 29-year-old gravida...  
1  She is now bleeding quite heavily. Ultrasound ...  
2  ALLERGIES: Heparin and Imitrex. PAST OB HISTOR...  
3  She had a pelvic ultrasound at Park Nicollet o...  
4  On _%#MMDD2007#%_, normal anatomy with anterio...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37500 entries, 

In [3]:
# Convert the 'Long_form' column of the dataframe into a list
labels = data['Long_form'].tolist()

# Convert the 'Context' column of the dataframe into a list
texts = data['Context'].tolist()

# Create a sorted list of unique labels
label_list = sorted(set(labels))

# Create a dictionary mapping each label to a unique index
label_map = {label: idx for idx, label in enumerate(label_list)}

# Create a dictionary to map indices back to labels for easier interpretation later
inverse_label_map = {idx: label for label, idx in label_map.items()}

# Print the mappings from labels to indices and vice versa
print("Label Map:", label_map)
print("Inverse Label Map:", inverse_label_map)

# Print the total number of unique labels
print("Total labels:", len(label_map))
print("Total Inverse Label:", len(inverse_label_map))

# Convert all labels in the original list to their corresponding indices
labels_indexed = [label_map[label] for label in labels]

# Import the train_test_split function from scikit-learn to split data into training and test sets
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets, with 20% of the data reserved as the test set
texts_train, texts_test, labels_train, labels_test = train_test_split(
    texts, labels_indexed, test_size=0.2, random_state=42)

Label Map: {'(PO Box) C4': 0, '(class) IA': 1, '(complement) component 3': 2, '(complement) component 4': 3, '(cycle) IB': 4, '(device) MP': 5, '(device) PD': 6, '(diltiazem) DC': 7, '(drug) AC': 8, '(drug) DC': 9, '(drug) DT': 10, '(drug) ES': 11, '(drug) IT': 12, '(drug) MP': 13, '(drug) MS': 14, '(drug) NP': 15, '(drug) OR': 16, '(drug) PAC': 17, '(drug) PD': 18, '(drug) PR': 19, '(drug) RT': 20, '(grade) IA': 21, '(grade) IB': 22, '(stage) C3': 23, '(stage) IA': 24, '(stage) IB': 25, '(status) IA': 26, '(status) IB': 27, '(type) IA': 28, '(type) IB': 29, 'American Society of Anesthesiologists': 30, 'American Society of Anesthesiologists:ASA': 31, 'BK (virus)': 32, 'C-reactive': 33, 'California': 34, "Children's Depression Inventory": 35, 'District of Columbia': 36, 'Fairview Southdale Hospital': 37, 'GENERAL ENGLISH': 38, 'Iowa': 39, 'Los Angeles': 40, 'Louisiana': 41, 'MISTAKE:EZ PAP': 42, 'MISTAKE:Oncotype DX': 43, 'MISTAKE:abduction': 44, 'Mall of America:MOA': 45, 'NAME': 46, '

In [4]:
print(f"Number of training samples: {len(texts_train)}")
print(f"Number of test samples: {len(texts_test)}")
print("Example of processed data:")
print(texts_train[0], labels_train[0])

Number of training samples: 30000
Number of test samples: 7500
Example of processed data:
Will get his home medications and continue those. Will ask neurology to review his antiepileptic regimen and have spoken with Dr. _%#NAME#%_. We will have OT and PT evaluate the patient. Suspect that he may need a TCU versus home with additional services. LABORATORY DATA: That was performed here includes the following laboratory tests. 267


In [6]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset
import torch

class ClinicalTextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Load the pretrained BioBERT Tokenizer and Model
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1', num_labels=len(label_map))

# Ensure all texts are strings before passing them to the tokenizer
texts_train = [str(text) for text in texts_train]
texts_test = [str(text) for text in texts_test]

# Encode the texts
train_encodings = tokenizer(texts_train, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(texts_test, truncation=True, padding=True, max_length=128)

# Create datasets
train_dataset = ClinicalTextDataset(train_encodings, labels_train)
test_dataset = ClinicalTextDataset(test_encodings, labels_test)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Load the pretrained BioBERT model without training
zero_shot_model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1', num_labels=len(label_map))
zero_shot_model.to(device)

# Create a DataLoader for the test dataset for zero-shot evaluation
zero_shot_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Initialize variables for tracking zero-shot accuracy
zero_shot_correct = 0
zero_shot_total = 0

# Evaluate the model without training
for batch in zero_shot_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = zero_shot_model(**batch)
    pred_indices = outputs.logits.argmax(-1).tolist()
    actual_indices = batch['labels'].tolist()
    zero_shot_correct += sum(p == a for p, a in zip(pred_indices, actual_indices))
    zero_shot_total += len(batch['labels'])

zero_shot_accuracy = (zero_shot_correct / zero_shot_total) * 100
print(f"Zero-shot accuracy before training: {zero_shot_accuracy:.2f}%")


# Start training
trainer.train()

# Save the fine-tuned model and tokenizer
model_path = "./biobert_finetuned"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# Verify the number of labels in the model configuration
print("Expected number of labels (from label map):", len(label_map))
print("Number of labels in model configuration:", model.config.num_labels)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
100,5.871700
200,5.582500
300,5.155700
400,4.858200
500,4.490800
600,3.948100
700,3.374100
800,2.853800
900,2.398800
1000,1.906700


Expected number of labels (from label map): 351
Number of labels in model configuration: 351


In [7]:
# Import required libraries
import torch
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader

# Set the device to GPU if available, otherwise fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pretrained BioBERT model with a specified number of classes
model_name = 'dmis-lab/biobert-v1.1'
trained_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_map))
trained_model.to(device)  # Move the model to the appropriate device

# Create a data loader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Initialize variables for tracking accuracy
correct_predictions = 0
total_predictions = 0

# Iterate over the test data
for batch in test_loader:
    # Move the batch data to the appropriate device
    batch = {k: v.to(device) for k, v in batch.items()}

    # Perform inference without computing gradients
    with torch.no_grad():
        outputs = trained_model(**batch)

    # Get the predicted class indices with the highest logit values
    pred_indices = outputs.logits.argmax(-1).tolist()

    # Compare predicted results with actual labels and count correct predictions
    actual_indices = batch['labels'].tolist()
    correct_predictions += sum(p == a for p, a in zip(pred_indices, actual_indices))
    total_predictions += len(batch['labels'])

# Calculate and print the accuracy of the model on the test data
accuracy = (correct_predictions / total_predictions) * 100
print(f"Accuracy of trained BioBERT model: {accuracy:.2f}%")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy of trained BioBERT model: 0.40%


In [8]:
# Create a data loader for the training dataset
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)  # Batch size can be adjusted based on GPU capacity

# Initialize variables for tracking accuracy
correct_predictions_train = 0
total_predictions_train = 0

# Iterate over the training data
for batch in train_loader:
    # Move the batch data to the appropriate device
    batch = {k: v.to(device) for k, v in batch.items()}

    # Perform inference without computing gradients
    with torch.no_grad():
        outputs = trained_model(**batch)

    # Get the predicted class indices with the highest logit values
    pred_indices = outputs.logits.argmax(-1).tolist()

    # Compare predicted results with actual labels and count correct predictions
    actual_indices = batch['labels'].tolist()
    correct_predictions_train += sum(p == a for p, a in zip(pred_indices, actual_indices))
    total_predictions_train += len(batch['labels'])

# Calculate and print the accuracy of the model on the training data
accuracy_train = (correct_predictions_train / total_predictions_train) * 100
print(f"Accuracy of trained model on training data: {accuracy_train:.2f}%")

Accuracy of trained model on training data: 0.36%
